## Import Dependencies

In [2]:
import os
import keras
import numpy as np
import imageio
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Activation, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
from keras import backend as K
import cv2
from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


#### Define our dataset class

In [ ]:
#Move validation images to training folder, allowing us to utilize train_test_split
from shutil import copyfile
         
for i in range(1,501):
    newindex = i + 1500
    copyfile("validation_images/validation_img_" + str(i) + ".jpg", "training_images/train_img_" + str(newindex) + ".jpg")
    copyfile("validation_masks/validation_mask_" + str(i) + ".jpg", "training_masks/train_mask_" + str(newindex) + ".jpg")

In [11]:
##Generate list of id's that will be used for file identification
train_ids = []
test_ids = []
for i in range(1,len(os.listdir('training_images'))):
    train_ids.append(i)  
    
for i in range(1,len(os.listdir('testing_images'))+1):
    test_ids.append(i)  

print(len(train_ids))
print(len(test_ids))

4000
927


In [12]:
#Specify paths for imagess and mask, and generate a collection of the file names.
TRAIN_PATH = 'training_images/'
TRAIN_MASK_PATH = 'training_masks/'
TEST_PATH = 'testing_images/'

train_ids2 = next(os.walk(TRAIN_PATH))[2]
mask_ids = next(os.walk(TRAIN_MASK_PATH))[2]
test_ids2 = next(os.walk(TEST_PATH))[2]

In [17]:
#Helper function to visualize image and masks
def plot2x2Array(image, mask):
    #invoke matplotlib!
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image)
    axarr[1].imshow(mask)
    axarr[0].grid()
    axarr[1].grid()
    axarr[0].set_title('Image')
    axarr[1].set_title('Mask')

In [15]:
#Specify dimensions of our input image.
im_width = 256
im_height = 256
im_chan = 3

In [ ]:
#Data Augmentation: Horizontal Flips only
for i in range(1,2001):
    img = cv2.imread('training_images/train_img_' + str(i) + '.jpg')
    mask = cv2.imread('training_masks/train_mask_' + str(i) + '.jpg')
    img = cv2.flip(img, 1)
    mask = cv2.flip(mask, 1)
    cv2.imwrite('training_images/train_img_' + str(i+2000) + '.jpg', img)
    cv2.imwrite('training_masks/train_mask_' + str(i+2000) + '.jpg', mask)    

## MODEL 1 (Basic U-NET Architecture)

In [ ]:
# Build Basic U-Net model

def unet(im_height, im_width, im_chan):
    input_img = Input((im_height, im_width, im_chan), name='img')

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (input_img)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[input_img], outputs=[outputs])
    return model

# Model 2 UNET 256x256

In [19]:
#Function creates Convolution block
def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

#Insert a max pooling layer to downsample
def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x


#Up sample the images followed by convolution
def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

#Function generates a U-net model with the above helper functions.
def get_unet2(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss=bce_dice_loss,
                  metrics=[dice_loss])
    
    return model

# Model 3 VGG16_U-NET Transfer Architecture

In [9]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dropout

#Import Pretained VGG16
base_pretrained_model = VGG16(input_shape =  (256,256,3), include_top = False, weights = 'imagenet')
base_pretrained_model.trainable = False
base_pretrained_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [11]:
#Add CONV2D Layer with Batch Normalization
x = base_pretrained_model.layers[-1].output
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='single_conv')(x)
x = (BatchNormalization())(x)

#Freeze first 5 convolution blocks
for layer in vggunet_custom.layers[:-3]:
    layer.trainable = False


In [12]:
#Prepare skip connections
f5 = vggunet_custom.get_layer("block5_conv1").output
f4 = vggunet_custom.get_layer("block4_conv1").output
f3 = vggunet_custom.get_layer("block3_conv1").output
f2 = vggunet_custom.get_layer("block2_conv1").output
f1 = vggunet_custom.get_layer("block1_conv1").output

#Upsample followed by CONV2D Transpose, Skip Connection, and finish the CONV block with Batch Normalization
o = UpSampling2D(size = (2,2))(x)
up6 = Conv2DTranspose(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(o)
merge6 = concatenate([f5, up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
o = (BatchNormalization())(conv6)

o = UpSampling2D(size = (2,2))(o)
up6 = Conv2DTranspose(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(o)
merge6 = concatenate([f4, up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
o = (BatchNormalization())(conv6)

o = UpSampling2D(size = (2,2))(o)
up6 = Conv2DTranspose(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(o)
merge6 = concatenate([f3, up6], axis = 3)
conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
o = (BatchNormalization())(conv6)

o = UpSampling2D(size = (2,2))(o)
up6 = Conv2DTranspose(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(o)
merge6 = concatenate([f2, up6], axis = 3)
conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
o = (BatchNormalization())(conv6)

o = UpSampling2D(size = (2,2))(o)
up6 = Conv2DTranspose(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(o)
merge6 = concatenate([f1, up6], axis = 3)
conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
o = (BatchNormalization())(conv6)

o = Conv2D(1, (1,1), activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal')(o)

vggunet_custom = Model(base_pretrained_model.input, o)

vggunet_custom.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [24]:
import sys
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.transform import resize

# Get and resize train images and masks
X = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.float32)
y = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.float32)

print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for i in range(1, len(train_ids) + 1):
    img = imread(TRAIN_PATH + 'train_img_' + str(i) + '.jpg')[:,:,:im_chan]
    x_img = resize(img, (im_height, im_width), mode='constant', preserve_range=True)
    X[i-1] = x_img / 255
    
    mask = img_to_array(load_img(TRAIN_MASK_PATH + 'train_mask_' + str(i) + '.jpg', grayscale=True))
    mask = resize(mask, (im_height, im_width, 1), mode='constant', preserve_range=True)
    y[i-1] = mask / 255

print('Done!')


Getting and resizing train images and masks ... 


C:\Users\user\Anaconda3\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


Done!


In [13]:
# Get and resize test images and prepare for prediction
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.float32)
sizes_test = []

print('Getting and resizing test images ... ')
sys.stdout.flush()

for n, id_ in tqdm(enumerate(test_ids2), total=len(test_ids2)):
    path = TEST_PATH + id_
    img = imread(path)[:,:,:im_chan]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (im_height, im_width), mode='constant', preserve_range=True)
    X_test[n] = img / 255

print('Done!')

Getting and resizing test images ... 


100%|████████████████████████████████████████| 927/927 [00:09<00:00, 94.48it/s]


Done!


## Without Augmentation

In [14]:
#Import train_test_split
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=42)

In [16]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + (1 - dice_loss(y_true, y_pred))

In [31]:
from keras.optimizers import SGD, Adam

#Define callbacks and begin training...
callbacks = [
    EarlyStopping(patience=15, verbose=1),
    ReduceLROnPlateau(patience=8, verbose=1),
    ModelCheckpoint('model-hairsg-vggunet_dropout.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

#results = model.fit(X_train, y_train, batch_size=64, epochs=500, callbacks=callbacks, validation_data=(X_valid, y_valid))
#vggunet_custom.compile(optimizer=SGD(lr=0.01), loss=bce_dice_loss, metrics=[dice_loss])
vggunet_custom.compile(optimizer=Adam(lr=1e-3), loss="binary_crossentropy", metrics=[dice_loss])
vggunet_custom.fit(X_train, y_train, batch_size=8, validation_data=(X_valid, y_valid), callbacks=callbacks, epochs=100)

Train on 3600 samples, validate on 401 samples
Epoch 1/100
1408/3600 [==========>...................] - ETA: 25:29 - loss: 0.0406 - dice_loss: 0.87 - ETA: 14:51 - loss: 0.0403 - dice_loss: 0.87 - ETA: 11:18 - loss: 0.0376 - dice_loss: 0.87 - ETA: 9:31 - loss: 0.0392 - dice_loss: 0.8745 - ETA: 8:27 - loss: 0.0394 - dice_loss: 0.874 - ETA: 7:44 - loss: 0.0423 - dice_loss: 0.876 - ETA: 7:13 - loss: 0.0432 - dice_loss: 0.876 - ETA: 6:50 - loss: 0.0441 - dice_loss: 0.873 - ETA: 6:32 - loss: 0.0434 - dice_loss: 0.873 - ETA: 6:17 - loss: 0.0439 - dice_loss: 0.870 - ETA: 6:05 - loss: 0.0424 - dice_loss: 0.872 - ETA: 5:55 - loss: 0.0427 - dice_loss: 0.871 - ETA: 5:46 - loss: 0.0428 - dice_loss: 0.870 - ETA: 5:39 - loss: 0.0422 - dice_loss: 0.871 - ETA: 5:32 - loss: 0.0423 - dice_loss: 0.869 - ETA: 5:26 - loss: 0.0443 - dice_loss: 0.867 - ETA: 5:21 - loss: 0.0440 - dice_loss: 0.866 - ETA: 5:16 - loss: 0.0432 - dice_loss: 0.865 - ETA: 5:12 - loss: 0.0430 - dice_loss: 0.865 - ETA: 5:08 - loss: 0.0

1424/3600 [==========>...................] - ETA: 4:20 - loss: 0.0354 - dice_loss: 0.879 - ETA: 4:21 - loss: 0.0437 - dice_loss: 0.882 - ETA: 4:20 - loss: 0.0469 - dice_loss: 0.871 - ETA: 4:20 - loss: 0.0496 - dice_loss: 0.864 - ETA: 4:20 - loss: 0.0497 - dice_loss: 0.870 - ETA: 4:20 - loss: 0.0495 - dice_loss: 0.869 - ETA: 4:19 - loss: 0.0489 - dice_loss: 0.869 - ETA: 4:18 - loss: 0.0471 - dice_loss: 0.872 - ETA: 4:18 - loss: 0.0487 - dice_loss: 0.875 - ETA: 4:17 - loss: 0.0470 - dice_loss: 0.877 - ETA: 4:16 - loss: 0.0467 - dice_loss: 0.875 - ETA: 4:15 - loss: 0.0481 - dice_loss: 0.873 - ETA: 4:14 - loss: 0.0477 - dice_loss: 0.874 - ETA: 4:14 - loss: 0.0471 - dice_loss: 0.873 - ETA: 4:13 - loss: 0.0479 - dice_loss: 0.871 - ETA: 4:12 - loss: 0.0477 - dice_loss: 0.870 - ETA: 4:12 - loss: 0.0471 - dice_loss: 0.868 - ETA: 4:11 - loss: 0.0465 - dice_loss: 0.868 - ETA: 4:10 - loss: 0.0463 - dice_loss: 0.866 - ETA: 4:10 - loss: 0.0462 - dice_loss: 0.866 - ETA: 4:09 - loss: 0.0461 - dice_los

3600/3600 [==============================] - ETA: 53s - loss: 0.0443 - dice_loss: 0.87 - ETA: 53s - loss: 0.0443 - dice_loss: 0.87 - ETA: 52s - loss: 0.0443 - dice_loss: 0.87 - ETA: 52s - loss: 0.0443 - dice_loss: 0.87 - ETA: 51s - loss: 0.0444 - dice_loss: 0.87 - ETA: 50s - loss: 0.0444 - dice_loss: 0.87 - ETA: 50s - loss: 0.0444 - dice_loss: 0.87 - ETA: 49s - loss: 0.0444 - dice_loss: 0.87 - ETA: 49s - loss: 0.0444 - dice_loss: 0.87 - ETA: 48s - loss: 0.0444 - dice_loss: 0.87 - ETA: 48s - loss: 0.0443 - dice_loss: 0.87 - ETA: 47s - loss: 0.0443 - dice_loss: 0.87 - ETA: 46s - loss: 0.0443 - dice_loss: 0.87 - ETA: 46s - loss: 0.0443 - dice_loss: 0.87 - ETA: 45s - loss: 0.0443 - dice_loss: 0.87 - ETA: 45s - loss: 0.0443 - dice_loss: 0.87 - ETA: 44s - loss: 0.0443 - dice_loss: 0.87 - ETA: 44s - loss: 0.0443 - dice_loss: 0.87 - ETA: 43s - loss: 0.0443 - dice_loss: 0.87 - ETA: 42s - loss: 0.0443 - dice_loss: 0.87 - ETA: 42s - loss: 0.0442 - dice_loss: 0.87 - ETA: 41s - loss: 0.0442 - dice_

1424/3600 [==========>...................] - ETA: 4:19 - loss: 0.0697 - dice_loss: 0.883 - ETA: 4:20 - loss: 0.0536 - dice_loss: 0.883 - ETA: 4:19 - loss: 0.0507 - dice_loss: 0.887 - ETA: 4:19 - loss: 0.0448 - dice_loss: 0.885 - ETA: 4:18 - loss: 0.0442 - dice_loss: 0.878 - ETA: 4:18 - loss: 0.0444 - dice_loss: 0.883 - ETA: 4:17 - loss: 0.0435 - dice_loss: 0.887 - ETA: 4:16 - loss: 0.0425 - dice_loss: 0.888 - ETA: 4:16 - loss: 0.0410 - dice_loss: 0.891 - ETA: 4:15 - loss: 0.0406 - dice_loss: 0.888 - ETA: 4:15 - loss: 0.0424 - dice_loss: 0.890 - ETA: 4:14 - loss: 0.0415 - dice_loss: 0.886 - ETA: 4:14 - loss: 0.0399 - dice_loss: 0.889 - ETA: 4:13 - loss: 0.0398 - dice_loss: 0.888 - ETA: 4:12 - loss: 0.0397 - dice_loss: 0.889 - ETA: 4:12 - loss: 0.0399 - dice_loss: 0.888 - ETA: 4:11 - loss: 0.0394 - dice_loss: 0.889 - ETA: 4:11 - loss: 0.0394 - dice_loss: 0.890 - ETA: 4:10 - loss: 0.0395 - dice_loss: 0.890 - ETA: 4:10 - loss: 0.0391 - dice_loss: 0.891 - ETA: 4:09 - loss: 0.0403 - dice_los

3600/3600 [==============================] - ETA: 54s - loss: 0.0390 - dice_loss: 0.88 - ETA: 53s - loss: 0.0390 - dice_loss: 0.88 - ETA: 52s - loss: 0.0390 - dice_loss: 0.88 - ETA: 52s - loss: 0.0390 - dice_loss: 0.88 - ETA: 51s - loss: 0.0390 - dice_loss: 0.88 - ETA: 51s - loss: 0.0390 - dice_loss: 0.88 - ETA: 50s - loss: 0.0390 - dice_loss: 0.88 - ETA: 50s - loss: 0.0391 - dice_loss: 0.88 - ETA: 49s - loss: 0.0391 - dice_loss: 0.88 - ETA: 48s - loss: 0.0391 - dice_loss: 0.88 - ETA: 48s - loss: 0.0391 - dice_loss: 0.88 - ETA: 47s - loss: 0.0390 - dice_loss: 0.88 - ETA: 47s - loss: 0.0390 - dice_loss: 0.88 - ETA: 46s - loss: 0.0390 - dice_loss: 0.88 - ETA: 45s - loss: 0.0391 - dice_loss: 0.88 - ETA: 45s - loss: 0.0391 - dice_loss: 0.88 - ETA: 44s - loss: 0.0391 - dice_loss: 0.88 - ETA: 44s - loss: 0.0391 - dice_loss: 0.88 - ETA: 43s - loss: 0.0391 - dice_loss: 0.88 - ETA: 43s - loss: 0.0391 - dice_loss: 0.88 - ETA: 42s - loss: 0.0391 - dice_loss: 0.88 - ETA: 41s - loss: 0.0391 - dice_

...

3600/3600 [==============================] - ETA: 54s - loss: 0.0259 - dice_loss: 0.92 - ETA: 53s - loss: 0.0259 - dice_loss: 0.92 - ETA: 52s - loss: 0.0259 - dice_loss: 0.92 - ETA: 52s - loss: 0.0259 - dice_loss: 0.92 - ETA: 51s - loss: 0.0259 - dice_loss: 0.92 - ETA: 51s - loss: 0.0259 - dice_loss: 0.92 - ETA: 50s - loss: 0.0259 - dice_loss: 0.92 - ETA: 49s - loss: 0.0259 - dice_loss: 0.92 - ETA: 49s - loss: 0.0259 - dice_loss: 0.92 - ETA: 48s - loss: 0.0259 - dice_loss: 0.92 - ETA: 48s - loss: 0.0259 - dice_loss: 0.92 - ETA: 47s - loss: 0.0259 - dice_loss: 0.92 - ETA: 47s - loss: 0.0259 - dice_loss: 0.92 - ETA: 46s - loss: 0.0259 - dice_loss: 0.92 - ETA: 45s - loss: 0.0259 - dice_loss: 0.92 - ETA: 45s - loss: 0.0258 - dice_loss: 0.92 - ETA: 44s - loss: 0.0259 - dice_loss: 0.92 - ETA: 44s - loss: 0.0259 - dice_loss: 0.92 - ETA: 43s - loss: 0.0258 - dice_loss: 0.92 - ETA: 42s - loss: 0.0258 - dice_loss: 0.92 - ETA: 42s - loss: 0.0258 - dice_loss: 0.92 - ETA: 41s - loss: 0.0258 - dice_

1424/3600 [==========>...................] - ETA: 4:19 - loss: 0.0373 - dice_loss: 0.935 - ETA: 4:20 - loss: 0.0310 - dice_loss: 0.936 - ETA: 4:19 - loss: 0.0256 - dice_loss: 0.929 - ETA: 4:18 - loss: 0.0263 - dice_loss: 0.932 - ETA: 4:18 - loss: 0.0263 - dice_loss: 0.933 - ETA: 4:17 - loss: 0.0248 - dice_loss: 0.934 - ETA: 4:17 - loss: 0.0245 - dice_loss: 0.932 - ETA: 4:16 - loss: 0.0244 - dice_loss: 0.931 - ETA: 4:16 - loss: 0.0245 - dice_loss: 0.930 - ETA: 4:15 - loss: 0.0246 - dice_loss: 0.930 - ETA: 4:15 - loss: 0.0249 - dice_loss: 0.929 - ETA: 4:14 - loss: 0.0251 - dice_loss: 0.929 - ETA: 4:13 - loss: 0.0250 - dice_loss: 0.928 - ETA: 4:13 - loss: 0.0251 - dice_loss: 0.928 - ETA: 4:12 - loss: 0.0247 - dice_loss: 0.928 - ETA: 4:12 - loss: 0.0247 - dice_loss: 0.928 - ETA: 4:11 - loss: 0.0244 - dice_loss: 0.927 - ETA: 4:11 - loss: 0.0241 - dice_loss: 0.927 - ETA: 4:10 - loss: 0.0242 - dice_loss: 0.927 - ETA: 4:09 - loss: 0.0242 - dice_loss: 0.926 - ETA: 4:09 - loss: 0.0240 - dice_los

3600/3600 [==============================] - ETA: 54s - loss: 0.0249 - dice_loss: 0.92 - ETA: 53s - loss: 0.0249 - dice_loss: 0.92 - ETA: 52s - loss: 0.0249 - dice_loss: 0.92 - ETA: 52s - loss: 0.0249 - dice_loss: 0.92 - ETA: 51s - loss: 0.0249 - dice_loss: 0.92 - ETA: 51s - loss: 0.0249 - dice_loss: 0.92 - ETA: 50s - loss: 0.0249 - dice_loss: 0.92 - ETA: 49s - loss: 0.0249 - dice_loss: 0.92 - ETA: 49s - loss: 0.0249 - dice_loss: 0.92 - ETA: 48s - loss: 0.0249 - dice_loss: 0.92 - ETA: 48s - loss: 0.0249 - dice_loss: 0.92 - ETA: 47s - loss: 0.0248 - dice_loss: 0.92 - ETA: 47s - loss: 0.0248 - dice_loss: 0.92 - ETA: 46s - loss: 0.0248 - dice_loss: 0.92 - ETA: 45s - loss: 0.0249 - dice_loss: 0.92 - ETA: 45s - loss: 0.0249 - dice_loss: 0.92 - ETA: 44s - loss: 0.0249 - dice_loss: 0.92 - ETA: 44s - loss: 0.0249 - dice_loss: 0.92 - ETA: 43s - loss: 0.0249 - dice_loss: 0.92 - ETA: 43s - loss: 0.0249 - dice_loss: 0.92 - ETA: 42s - loss: 0.0249 - dice_loss: 0.92 - ETA: 41s - loss: 0.0249 - dice_

1424/3600 [==========>...................] - ETA: 4:20 - loss: 0.0198 - dice_loss: 0.934 - ETA: 4:20 - loss: 0.0239 - dice_loss: 0.928 - ETA: 4:19 - loss: 0.0225 - dice_loss: 0.930 - ETA: 4:19 - loss: 0.0227 - dice_loss: 0.932 - ETA: 4:18 - loss: 0.0215 - dice_loss: 0.934 - ETA: 4:17 - loss: 0.0216 - dice_loss: 0.934 - ETA: 4:17 - loss: 0.0218 - dice_loss: 0.929 - ETA: 4:16 - loss: 0.0226 - dice_loss: 0.929 - ETA: 4:16 - loss: 0.0231 - dice_loss: 0.928 - ETA: 4:15 - loss: 0.0227 - dice_loss: 0.929 - ETA: 4:15 - loss: 0.0226 - dice_loss: 0.929 - ETA: 4:14 - loss: 0.0233 - dice_loss: 0.930 - ETA: 4:13 - loss: 0.0229 - dice_loss: 0.930 - ETA: 4:13 - loss: 0.0231 - dice_loss: 0.930 - ETA: 4:12 - loss: 0.0231 - dice_loss: 0.931 - ETA: 4:12 - loss: 0.0226 - dice_loss: 0.931 - ETA: 4:11 - loss: 0.0228 - dice_loss: 0.931 - ETA: 4:10 - loss: 0.0226 - dice_loss: 0.931 - ETA: 4:10 - loss: 0.0229 - dice_loss: 0.931 - ETA: 4:09 - loss: 0.0228 - dice_loss: 0.931 - ETA: 4:09 - loss: 0.0229 - dice_los

3600/3600 [==============================] - ETA: 54s - loss: 0.0239 - dice_loss: 0.92 - ETA: 53s - loss: 0.0239 - dice_loss: 0.92 - ETA: 52s - loss: 0.0239 - dice_loss: 0.92 - ETA: 52s - loss: 0.0239 - dice_loss: 0.92 - ETA: 51s - loss: 0.0239 - dice_loss: 0.92 - ETA: 51s - loss: 0.0239 - dice_loss: 0.92 - ETA: 50s - loss: 0.0239 - dice_loss: 0.92 - ETA: 49s - loss: 0.0239 - dice_loss: 0.92 - ETA: 49s - loss: 0.0239 - dice_loss: 0.92 - ETA: 48s - loss: 0.0239 - dice_loss: 0.92 - ETA: 48s - loss: 0.0239 - dice_loss: 0.92 - ETA: 47s - loss: 0.0239 - dice_loss: 0.92 - ETA: 47s - loss: 0.0239 - dice_loss: 0.92 - ETA: 46s - loss: 0.0239 - dice_loss: 0.92 - ETA: 45s - loss: 0.0239 - dice_loss: 0.92 - ETA: 45s - loss: 0.0239 - dice_loss: 0.92 - ETA: 44s - loss: 0.0239 - dice_loss: 0.92 - ETA: 44s - loss: 0.0239 - dice_loss: 0.92 - ETA: 43s - loss: 0.0239 - dice_loss: 0.92 - ETA: 43s - loss: 0.0239 - dice_loss: 0.92 - ETA: 42s - loss: 0.0239 - dice_loss: 0.92 - ETA: 41s - loss: 0.0239 - dice_

# Evaluation

In [38]:
#Load best weights
vggunet_custom.load_weights('model-hairsg-vggunet_dropout.h5')

In [39]:
#Evaluate model
from keras.optimizers import Adam
vggunet_custom.compile(optimizer=Adam(lr=0.01), loss=bce_dice_loss, metrics=[dice_loss])
vggunet_custom.evaluate(X_valid, y_valid)

401/401 [==============================] - ETA: 15 - ETA: 11 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 11s 28ms/step


[0.21501746584203774, 0.8509822030019879]

In [40]:
#Make predictions
preds_test = vggunet_custom.predict(X_test, verbose=1)
#Remove less certain predictions
preds_test_t = (preds_test > 0.5).astype(np.uint8)

927/927 [==============================] - ETA: 31 - ETA: 26 - ETA: 24 - ETA: 22 - ETA: 21 - ETA: 20 - ETA: 19 - ETA: 18 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 25s 27ms/step


In [44]:
#Save images and prepare for RLE
import scipy.misc
    
for n, id_ in tqdm(enumerate(test_ids2), total=len(test_ids2)):
    img = resize(np.squeeze(preds_test[n]), (sizes_test[n][0], sizes_test[n][1]), mode='constant')
    last = id_.split('_')[2]
    scipy.misc.imsave('test_masks/test_mask_' + last , img)

  0%|                                                  | 0/927 [00:00<?, ?it/s]C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  
100%|███████████████████████████████████████| 927/927 [00:06<00:00, 143.56it/s]


In [45]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from skimage.transform import resize
from skimage.io import imread
%matplotlib inline  

# encoding function
# based on the implementation: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python/code
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

In [46]:
# (* update) the input_path using your folder path
input_path = './test_masks'

# get a sorted list of all mask filenames in the folder
masks = [f for f in os.listdir(input_path) if f.endswith('.jpg')]
masks = sorted(masks, key=lambda s:int(s.split('_')[2].split('.')[0]))

# encode all masks
encodings = []
for file in masks:
    mask = imread(os.path.join(input_path, file))
    #img_size =10
    #mask = resize(mask, (img_size, img_size), mode='constant', preserve_range=True)
    mask = np.array(mask, dtype=np.uint8)
    mask = np.round(mask/255)
    encodings.append(rle_encoding(mask))


# (** update) the path where to save the submission csv file
sub = pd.DataFrame()
sub['ImageId'] = pd.Series(masks).apply(lambda x: os.path.splitext(x)[0])
sub['EncodedPixels'] = pd.Series(encodings).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv(os.path.join('.', 'vgg16unet_t5.csv'), index=False)